Theoretical Questions

Exercise 1:
⊛ OLS is prone to overfitting in high-dimensional datasets. Also, high dimension datasets exhibit multicolinearity. 
⊛ Using regularization methods like ridge regression would be better. This is because adding regularization terms can help prevent overfitting by discouraging large coefficients. 

Exercise 2:
(c) The objective function is the same as ordinary least squares objective function.

Exercise 3: 
(i) (b) and (c)

Exercise 4:
High Bias. Reduce regularization parameter.


# Coding Question

In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook,tqdm

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error


# Exercise 5
car_price=pd.read_csv('CarPrice_Assignment.csv')
X=car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower',
 'peakrpm', 'citympg', 'highwaympg']]
y=car_price['price']
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2)
lasso_coef=[]
for i in tqdm(range(0,1000)):
    kf=KFold(n_splits=5, shuffle=True, random_state=i)
    lasso_cv=Pipeline([('scaler', StandardScaler()),
                       ('model', LassoCV(cv=kf.split(X_train)))]).fit(X_train, y_train)
    lasso_lambda=lasso_cv.named_steps['model'].alpha_
    lasso_md=Pipeline([('scaler', StandardScaler()),
                       ('model', Lasso(alpha=lasso_lambda))]).fit(X_train, y_train)
    lasso_coef.append(lasso_md.named_steps['model'].coef_)
lasso_coef=pd.DataFrame(lasso_coef)
lasso_coef.columns=X.columns.tolist()
lasso_coef.head()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:20<00:00, 12.47it/s]


,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg,highwaympg
0,871.127823,4083.415887,1419.873744,1938.712928,878.285879,-977.016240,-0.0
1,876.472967,4116.003950,1459.438358,1909.822159,924.791599,-1002.141233,-0.0
2,880.634502,4148.938129,1496.141084,1880.538457,968.546118,-1025.868121,0.0
3,874.627688,4104.096602,1445.216746,1920.472712,908.048675,-993.011160,-0.0
4,876.472967,4116.003950,1459.438358,1909.822159,924.791599,-1002.141233,-0.0


In [33]:
(lasso_coef==0).sum(axis=0)

wheelbase             0
enginesize            0
compressionratio      0
horsepower            0
peakrpm               0
citympg               0
highwaympg          907
dtype: int64

In [34]:
X_train=X_train.drop(columns='highwaympg',axis=1)
X_test=X_test.drop(columns='highwaympg',axis=1)

lm_md = make_pipeline(StandardScaler(), LinearRegression()).fit(X_train,y_train)
lm_pred=lm_md.predict(X_test)
lm_mse=mean_squared_error(y_test, lm_pred)
print('The linear model mse is', lm_mse)

The linear model mse is 13316485.749820003


In [35]:
X_test.head()

,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg
198,104.3,130,7.5,162,5100,17
106,99.2,181,9.0,160,5200,19
199,104.3,130,7.5,162,5100,17
124,95.9,156,7.0,145,5000,19
26,93.7,90,9.4,68,5500,31


In [36]:
y_test.head()

198    18420.0
106    18399.0
199    18950.0
124    12764.0
26      7609.0
Name: price, dtype: float64

In [37]:
ridge_lambda=list()

for i in tqdm(range(0,100)):
    kf=KFold(n_splits=5, shuffle=True, random_state=i)
    ridge_cv=Pipeline([('scaler', StandardScaler()),
                       ('model', RidgeCV(alphas=[0.001, 0.01, 0.1, 1, 10, 100],
                                        cv=kf.split(X_train)))]).fit(X_train, y_train)
    ridge_lambda.append(ridge_cv.named_steps['model'].alpha_)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:06<00:00, 16.31it/s]


In [38]:
pd.DataFrame(ridge_lambda).value_counts()

10.0    85
1.0     14
0.1      1
Name: count, dtype: int64

In [41]:
ridge_md=make_pipeline(StandardScaler(), Ridge(alpha=10)).fit(X_train,y_train)
ridge_pred=ridge_md.predict(X_test)
ridge_mse=mean_squared_error(y_test, ridge_pred)
print('The Ridge model mse is', ridge_mse)

The Ridge model mse is 13668938.46101259


In [42]:
if lm_mse < ridge_mse:
    print("I would use Linear Regression to predict car prices because it has a lower MSE.")
else:
    print("I would use Ridge Regression to predict car prices because it has a lower MSE.")

I would use Linear Regression to predict car prices because it has a lower MSE.
